In [2]:
import concurrent.futures as cf
import glob
import io
import os
import time
from pathlib import Path
from tempfile import NamedTemporaryFile
from typing import List, Literal

from dotenv import load_dotenv
from loguru import logger
from openai import OpenAI
from promptic import llm
from pydantic import BaseModel, ValidationError
from pypdf import PdfReader
from tenacity import retry, retry_if_exception_type

# Load environment variables

In [3]:
load_dotenv()
# Set base dir equal to parent directory 
notebook_path = Path().resolve()  # Gets the current working directory
if notebook_path.name != os.getenv("ROOT_DIR"):
    os.chdir(notebook_path.parent)
print(Path().resolve())
if os.getenv("OPENAI_API_KEY")  and os.getenv("GEMINI_API_KEY"):        
    print("API keys loaded")
else:
    print("WARNING: API keys not found")

/Users/ekabrams_work/Documents/cursor_projects/my-audio-summaries
API keys loaded


# Core Functions

In [6]:
example_list=[str(p) for p in Path(BASE_DIR / "examples").glob("*.pdf")]
example = example_list[0]
generate_audio(example)


2025-01-07 16:57:05.927 | INFO     | __main__:generate_audio:101 - Generated 3313 characters of audio


('./sandbox_examples/audio/Gene-therapy-for-deafness_2025-01-07_16-57-05.mp3',
 'female-1: Welcome to "BioBreakthroughs," the podcast that explores the amazing advancements in the world of biology and medicine. Today, we\'re diving deep into a truly remarkable story of hope and healing.\n\nfemale-1: And I\'m thrilled to be joined by Dr. Anya Sharma, a leading geneticist who\'s been at the forefront of this incredible work.\n\nfemale-2: It\'s a pleasure to be here. Thanks for having me.\n\nfemale-1: Dr. Sharma, let\'s talk about this groundbreaking gene therapy for hearing loss.  I\'ve heard stories about children born deaf who are now hearing, and it\'s absolutely incredible.\n\nfemale-2: It is. We\'ve made significant strides in treating DFNB9, a type of hereditary deafness that affects about 2-8% of people born with hearing loss. Imagine your ears are a beautifully complex machine, and the OTOF gene is like a tiny, essential gear.  In DFNB9, this gear is broken, hindering the transmi

# Split main function up into smaller functions

In [51]:
file = example_list[0]
with Path(file).open("rb") as f:
        reader = PdfReader(f)
        print(type(reader.pages))
        tmp = reader.pages[0]
        text = "\n\n".join([page.extract_text() for page in reader.pages])

print(text[:500])
text1 = text[:500]

<class 'pypdf._page._VirtualList'>
Cosmos » Biology
Gene therapy restores hearing to children with inherited deafness
Credit: Nick Dolding/Getty Images
 e  rst clinical trial to administer gene therapy to both ears in one person has restored hearing function to 5
children born with a form of inherited deafness, astounding the research team..
Two of the children even gained an ability to appreciate music.
 e success of the new approach is detailed in a new study published in Nature M edicin e.  e work builds on the  rst
phase of t


In [1]:
import asyncio
from io import BytesIO
from src.test_gmail_fetch import test_fetch

# Assuming you have your PDF as bytes in a variable called pdf_bytes
async def run_fetch():
    return await test_fetch(verbose=True)

documents = await run_fetch()
print(f'{len(documents)} found, with titles:')
print('\n'.join(doc.title for doc in documents))


In [10]:
from googleapiclient.discovery import build
import pickle
from datetime import datetime, timedelta
import os


# Convert datetime to Gmail query format (YYYY/MM/DD)
since = datetime.now() - timedelta(days=7)
date_str = since.strftime("%Y/%m/%d")
# query = f"has:attachment after:{date_str}"
query = f"after:{date_str}"

print(f"Searching with query: {query}")

if os.path.exists('credentials/token.pickle'):
        with open('credentials/token.pickle', 'rb') as token:
            creds = pickle.load(token)

service = build('gmail', 'v1', credentials=creds)

# Get list of messages matching our query
results = service.users().messages().list(
    userId='me',
    q=query
    ).execute()

messages = results.get('messages', [])
print(f"Found {len(messages)} emails with attachments")

Searching with query: after:2025/01/01
Found 9 emails with attachments


In [32]:
import json
def get_parts(payload):  
        """Recursively get all parts from the email payload"""
        parts = []
        
        # If this part has parts, process them
        if 'parts' in payload:
            for part in payload['parts']:
                parts.extend(get_parts(part))
        
        # Add this part if it's not a container
        if payload.get('filename'):
            parts.append(payload)
            
        return parts

message = messages[1]
message_id = message['id']

message_data = service.users().messages().get(userId='me', id=message_id).execute()
parts = get_parts(message_data['payload'])

print(parts)

print(json.dumps(parts[1], indent=2))

[]


IndexError: list index out of range